# LangGraph Basics: Graph API vs Functional API

This notebook demonstrates LangGraph concepts using **both APIs**:
- **Graph API (StateGraph)**: Explicit, declarative graph construction
- **Functional API**: Imperative, traditional programming approach

## Learning Objectives
1. Build the same workflows using both APIs
2. Understand when to use each approach
3. Compare code structure and complexity
4. Learn the strengths of each paradigm

## Labs Covered
1. **Lab 1**: Linear Workflow - Employee Onboarding
2. **Lab 2**: Conditional Routing - Leave Approval System

## Setup and Installation

In [ ]:
# Install required packages
!pip install --pre -qU langgraph langchain langchain-openai

In [ ]:
# Import required libraries
import os
from typing import TypedDict, Annotated, Literal
from langchain_openai import ChatOpenAI

# Configure API key (use your method: Colab secrets, env vars, etc.)
# For Colab:
# from google.colab import userdata
# os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

# Or directly:
os.environ['OPENAI_API_KEY'] = 'your-api-key-here'

# Initialize LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

print("✅ Setup complete!")

---
# Lab 1: Employee Onboarding Workflow

**Scenario**: Create a linear workflow that onboards new employees through these steps:
1. Validate employee information
2. Assign equipment based on department
3. Set up accounts
4. Send welcome email

We'll build this **twice** - once with each API.

## Approach 1: Using Graph API (StateGraph)

### Step 1: Define State

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

# Define state schema
class OnboardingState(TypedDict):
    """State for employee onboarding workflow"""
    employee_name: str
    employee_id: str
    department: str
    messages: Annotated[list, add_messages]
    validation_status: str
    tasks_completed: list

print("✅ Graph API: State defined")

### Step 2: Define Nodes (Functions)

In [ ]:
def validate_employee_graph(state: OnboardingState):
    """Node 1: Validate employee information"""
    print(f"📋 [Graph API] Validating {state['employee_name']}...")
    
    if state['employee_name'] and state['employee_id']:
        return {
            "validation_status": "✅ Validated",
            "messages": [("assistant", f"Employee {state['employee_name']} validated")]
        }
    return {"validation_status": "❌ Failed"}

def assign_equipment_graph(state: OnboardingState):
    """Node 2: Assign equipment"""
    print(f"💻 [Graph API] Assigning equipment for {state['department']}...")
    
    equipment = {
        "Engineering": ["MacBook Pro", "Monitor"],
        "HR": ["Laptop", "Headset"],
        "Sales": ["MacBook Air", "Phone"],
    }
    
    return {
        "tasks_completed": state['tasks_completed'] + ["Equipment assigned"],
        "messages": [("assistant", f"Assigned: {', '.join(equipment.get(state['department'], ['Laptop']))}")]
    }

def setup_accounts_graph(state: OnboardingState):
    """Node 3: Setup accounts"""
    print(f"🔐 [Graph API] Setting up accounts...")
    
    return {
        "tasks_completed": state['tasks_completed'] + ["Accounts created"],
        "messages": [("assistant", "Created: Email, Slack, HR Portal")]
    }

def send_welcome_graph(state: OnboardingState):
    """Node 4: Send welcome email"""
    print(f"📧 [Graph API] Sending welcome email...")
    
    return {
        "tasks_completed": state['tasks_completed'] + ["Welcome email sent"],
        "messages": [("assistant", f"Welcome email sent to {state['employee_name']}")]
    }

print("✅ Graph API: All nodes defined")

### Step 3: Build the Graph

In [ ]:
# Create graph
workflow_graph = StateGraph(OnboardingState)

# Add nodes
workflow_graph.add_node("validate", validate_employee_graph)
workflow_graph.add_node("assign_equipment", assign_equipment_graph)
workflow_graph.add_node("setup_accounts", setup_accounts_graph)
workflow_graph.add_node("send_welcome", send_welcome_graph)

# Add edges (define the flow)
workflow_graph.add_edge(START, "validate")
workflow_graph.add_edge("validate", "assign_equipment")
workflow_graph.add_edge("assign_equipment", "setup_accounts")
workflow_graph.add_edge("setup_accounts", "send_welcome")
workflow_graph.add_edge("send_welcome", END)

# Compile
app_graph = workflow_graph.compile()

print("✅ Graph API: Workflow compiled")
print("Flow: START → validate → assign → setup → welcome → END")

### Step 4: Run the Graph API Workflow

In [ ]:
print("\n" + "="*70)
print("GRAPH API: Running Onboarding Workflow")
print("="*70)

initial_state = {
    "employee_name": "Karan Singh",
    "employee_id": "106",
    "department": "Engineering",
    "messages": [],
    "validation_status": "",
    "tasks_completed": []
}

result_graph = app_graph.invoke(initial_state)

print("\n" + "="*70)
print("RESULT")
print("="*70)
print(f"Status: {result_graph['validation_status']}")
print(f"Tasks: {len(result_graph['tasks_completed'])}")
for task in result_graph['tasks_completed']:
    print(f"  ✓ {task}")

## Approach 2: Using Functional API

Now let's build the **same workflow** using the Functional API.

### Key Differences:
- No explicit State class needed
- Use `@entrypoint` and `@task` decorators
- Standard Python control flow (no graph construction)
- State is scoped to functions

In [ ]:
from langgraph.func import entrypoint, task

# Define tasks (similar to nodes, but with @task decorator)
@task()
async def validate_employee_func(name: str, emp_id: str):
    """Task 1: Validate employee"""
    print(f"📋 [Functional API] Validating {name}...")
    
    if name and emp_id:
        return {"status": "✅ Validated", "message": f"Employee {name} validated"}
    return {"status": "❌ Failed"}

@task()
async def assign_equipment_func(department: str):
    """Task 2: Assign equipment"""
    print(f"💻 [Functional API] Assigning equipment for {department}...")
    
    equipment = {
        "Engineering": ["MacBook Pro", "Monitor"],
        "HR": ["Laptop", "Headset"],
        "Sales": ["MacBook Air", "Phone"],
    }
    
    return {"equipment": equipment.get(department, ["Laptop"])}

@task()
async def setup_accounts_func(name: str):
    """Task 3: Setup accounts"""
    print(f"🔐 [Functional API] Setting up accounts for {name}...")
    
    return {"accounts": ["Email", "Slack", "HR Portal"]}

@task()
async def send_welcome_func(name: str):
    """Task 4: Send welcome email"""
    print(f"📧 [Functional API] Sending welcome email...")
    
    return {"email_sent": True, "message": f"Welcome email sent to {name}"}

# Define the entrypoint (main workflow)
@entrypoint()
async def onboard_employee_func(employee_name: str, employee_id: str, department: str):
    """
    Main workflow using Functional API
    Notice: No graph construction, just regular Python!
    """
    
    # Step 1: Validate
    validation = await validate_employee_func(employee_name, employee_id)
    
    if validation["status"] == "❌ Failed":
        return {"error": "Validation failed"}
    
    # Step 2: Assign equipment
    equipment = await assign_equipment_func(department)
    
    # Step 3: Setup accounts
    accounts = await setup_accounts_func(employee_name)
    
    # Step 4: Send welcome
    welcome = await send_welcome_func(employee_name)
    
    # Return final result
    return {
        "validation_status": validation["status"],
        "equipment_assigned": equipment["equipment"],
        "accounts_created": accounts["accounts"],
        "welcome_sent": welcome["email_sent"],
        "tasks_completed": 4
    }

print("✅ Functional API: Workflow defined")

### Run the Functional API Workflow

In [ ]:
print("\n" + "="*70)
print("FUNCTIONAL API: Running Onboarding Workflow")
print("="*70)

# Run the workflow
result_func = await onboard_employee_func(
    employee_name="Karan Singh",
    employee_id="106",
    department="Engineering"
)

print("\n" + "="*70)
print("RESULT")
print("="*70)
print(f"Status: {result_func['validation_status']}")
print(f"Equipment: {', '.join(result_func['equipment_assigned'])}")
print(f"Accounts: {', '.join(result_func['accounts_created'])}")
print(f"Tasks completed: {result_func['tasks_completed']}")

## 🔍 Lab 1 Comparison

### Graph API
**Pros:**
- ✅ Explicit visualization of workflow
- ✅ Easy to modify flow by changing edges
- ✅ Better for complex multi-actor systems
- ✅ State transitions are explicit

**Cons:**
- ❌ More boilerplate code
- ❌ Need to define State schema
- ❌ More verbose for simple workflows

### Functional API
**Pros:**
- ✅ Less boilerplate, cleaner code
- ✅ Familiar Python programming style
- ✅ Easier for simple linear workflows
- ✅ No explicit state management needed

**Cons:**
- ❌ Flow is implicit in code
- ❌ Harder to visualize
- ❌ Less suitable for complex branching

### When to Use Which?
- **Graph API**: Complex workflows, multi-agent systems, need visualization
- **Functional API**: Simple workflows, rapid prototyping, prefer imperative style

---
# Lab 2: Conditional Routing - Leave Approval

**Scenario**: Route leave requests based on conditions:
- Insufficient balance → Reject
- ≤3 days + sufficient balance → Auto-approve
- >3 days + sufficient balance → Manager review

This demonstrates **conditional edges** (Graph API) vs **if/else** (Functional API).

## Setup: Employee Database

In [ ]:
# Employee data
employees = {
    "101": {"name": "Priya Sharma", "department": "Engineering", "manager": "102"},
    "102": {"name": "Rahul Verma", "department": "Engineering", "manager": "103"},
    "103": {"name": "Anjali Patel", "department": "HR", "manager": None},
}

leave_balances = {"101": 12, "102": 8, "103": 15}

print("✅ Employee database loaded")

## Approach 1: Graph API with Conditional Edges

In [ ]:
# Define state
class LeaveState(TypedDict):
    employee_id: str
    employee_name: str
    days_requested: int
    leave_balance: int
    approval_status: str
    messages: Annotated[list, add_messages]

# Define nodes
def check_balance_graph(state: LeaveState):
    print(f"\n📊 [Graph API] Checking balance for {state['employee_name']}...")
    balance = leave_balances.get(state['employee_id'], 0)
    return {"leave_balance": balance}

def auto_approve_graph(state: LeaveState):
    print(f"✅ [Graph API] Auto-approved!")
    return {
        "approval_status": "✅ Auto-Approved",
        "messages": [("assistant", "Leave approved automatically")]
    }

def manager_review_graph(state: LeaveState):
    print(f"👔 [Graph API] Sent to manager...")
    return {
        "approval_status": "⏳ Pending Manager Review",
        "messages": [("assistant", "Sent to manager for approval")]
    }

def reject_graph(state: LeaveState):
    print(f"❌ [Graph API] Rejected!")
    return {
        "approval_status": "❌ Rejected - Insufficient Balance",
        "messages": [("assistant", "Request denied due to insufficient balance")]
    }

# Routing function - KEY CONCEPT!
def route_leave_graph(state: LeaveState) -> Literal["approve", "manager", "reject"]:
    """
    This function decides which node to go to next!
    """
    days = state['days_requested']
    balance = state['leave_balance']
    
    # Check balance first
    if days > balance:
        return "reject"
    
    # Short leaves auto-approved
    if days <= 3:
        return "approve"
    
    # Long leaves need manager approval
    return "manager"

# Build graph
leave_workflow = StateGraph(LeaveState)

# Add nodes
leave_workflow.add_node("check_balance", check_balance_graph)
leave_workflow.add_node("approve", auto_approve_graph)
leave_workflow.add_node("manager", manager_review_graph)
leave_workflow.add_node("reject", reject_graph)

# Add edges
leave_workflow.add_edge(START, "check_balance")

# Add CONDITIONAL edge - this is the key!
leave_workflow.add_conditional_edges(
    "check_balance",
    route_leave_graph,
    {
        "approve": "approve",
        "manager": "manager",
        "reject": "reject"
    }
)

# All routes end
leave_workflow.add_edge("approve", END)
leave_workflow.add_edge("manager", END)
leave_workflow.add_edge("reject", END)

# Compile
leave_app_graph = leave_workflow.compile()

print("✅ Graph API: Conditional workflow compiled")

### Test Graph API - Different Scenarios

In [ ]:
test_cases_graph = [
    {"employee_id": "101", "employee_name": "Priya", "days_requested": 2},  # Auto-approve
    {"employee_id": "102", "employee_name": "Rahul", "days_requested": 5}, # Manager review
    {"employee_id": "103", "employee_name": "Anjali", "days_requested": 20}, # Reject
]

print("\n" + "="*70)
print("GRAPH API: Testing Leave Approval")
print("="*70)

for test in test_cases_graph:
    initial = {**test, "leave_balance": 0, "approval_status": "", "messages": []}
    result = leave_app_graph.invoke(initial)
    print(f"\n{result['employee_name']} ({result['days_requested']} days): {result['approval_status']}")

## Approach 2: Functional API with If/Else

In [ ]:
from langgraph.func import entrypoint, task

# Tasks
@task()
async def check_balance_func(employee_id: str):
    print(f"📊 [Functional API] Checking balance...")
    return leave_balances.get(employee_id, 0)

@task()
async def auto_approve_func():
    print(f"✅ [Functional API] Auto-approved!")
    return "✅ Auto-Approved"

@task()
async def manager_review_func():
    print(f"👔 [Functional API] Sent to manager...")
    return "⏳ Pending Manager Review"

@task()
async def reject_func():
    print(f"❌ [Functional API] Rejected!")
    return "❌ Rejected - Insufficient Balance"

# Entrypoint with standard if/else logic
@entrypoint()
async def approve_leave_func(employee_id: str, employee_name: str, days_requested: int):
    """
    Leave approval using Functional API
    Notice: Just regular Python if/else - no routing function needed!
    """
    
    # Check balance
    balance = await check_balance_func(employee_id)
    
    # Routing logic with standard Python
    if days_requested > balance:
        status = await reject_func()
    elif days_requested <= 3:
        status = await auto_approve_func()
    else:
        status = await manager_review_func()
    
    return {
        "employee_name": employee_name,
        "days_requested": days_requested,
        "balance": balance,
        "approval_status": status
    }

print("✅ Functional API: Leave approval workflow defined")

### Test Functional API - Same Scenarios

In [ ]:
test_cases_func = [
    {"employee_id": "101", "employee_name": "Priya", "days_requested": 2},
    {"employee_id": "102", "employee_name": "Rahul", "days_requested": 5},
    {"employee_id": "103", "employee_name": "Anjali", "days_requested": 20},
]

print("\n" + "="*70)
print("FUNCTIONAL API: Testing Leave Approval")
print("="*70)

for test in test_cases_func:
    result = await approve_leave_func(**test)
    print(f"\n{result['employee_name']} ({result['days_requested']} days): {result['approval_status']}")

## 🔍 Lab 2 Comparison

### Conditional Logic

**Graph API:**
```python
# Routing function
def route(state) -> Literal["a", "b", "c"]:
    if condition1:
        return "a"
    return "b"

# Add conditional edge
graph.add_conditional_edges("node", route, {"a": "node_a", "b": "node_b"})
```

**Functional API:**
```python
@entrypoint()
async def workflow():
    if condition1:
        result = await task_a()
    else:
        result = await task_b()
    return result
```

### Which is Better?

**Graph API wins when:**
- Complex branching with many paths
- Need to visualize decision flow
- Multiple conditional routes
- Team needs to understand flow at a glance

**Functional API wins when:**
- Simple if/else logic
- Prefer standard Python patterns
- Want less boilerplate
- Rapid prototyping

## 🎯 Exercises

### Exercise 1: Add Notification Step
Add a notification step to both workflows that:
- Sends email for all approved leaves
- Sends SMS for manager review cases

**Hint for Graph API:** Add a new node and conditional edge after approval nodes
**Hint for Functional API:** Add if/else after status determination

### Exercise 2: Emergency Leave
Add emergency leave handling:
- If reason contains "emergency", always approve (regardless of balance)
- Add "reason" field to state

Implement in both APIs and compare complexity.

### Exercise 3: Multi-Stage Approval
For leaves >7 days, require:
1. Manager approval
2. Then HR approval

Implement in both APIs. Which one is easier?

### Exercise 4: Convert Graph to Functional
Take this Graph API workflow and convert to Functional API:
```python
# START → validate → [pass: process, fail: reject] → END
```

### Exercise 5: Convert Functional to Graph
Take this Functional API workflow and convert to Graph API:
```python
@entrypoint()
async def workflow():
    data = await fetch()
    if data["priority"] == "high":
        await urgent_process()
    else:
        await normal_process()
```

## 📊 Final Comparison Summary

| Aspect | Graph API | Functional API |
|--------|-----------|----------------|
| **Paradigm** | Declarative | Imperative |
| **State Management** | Explicit TypedDict | Implicit (function scope) |
| **Control Flow** | Nodes + Edges | Standard Python |
| **Visualization** | ✅ Easy | ❌ Harder |
| **Code Lines** | More | Less |
| **Learning Curve** | Steeper | Gentler |
| **Best For** | Complex systems | Simple workflows |
| **Debugging** | Node-by-node | Standard debugging |
| **Flexibility** | Very high | High |

## 🎓 Key Takeaways

1. **Both APIs are powerful** - choose based on your needs
2. **Graph API excels** at complex, multi-actor systems
3. **Functional API excels** at rapid development and simple flows
4. **You can mix both** - use graphs from entrypoints, tasks from nodes
5. **Start simple** - begin with Functional, move to Graph when needed

## 🚀 Next Steps

- Build a ReAct agent with both APIs
- Explore human-in-the-loop with both approaches
- Learn about persistence and streaming
- Dive into multi-agent systems